## DLI Tutorial on ColossalAI Framework

This tutorial serves as a gentle introduction into using ColossalAI for large-scale training and achieving multiple types of parallelism. There are in total 8 sections detailing the steps to realize:

1. Environment setup; <br>
2. Large-scale optimizer; <br>
3. Hybrid parallelism; <br>
4. Sequence parallelism; <br>
5. Auto parallelism; <br> 
6. OPT model inference; <br>
7. Fastfold inference; <br>
8. Stable diffusion inference. <br>

Run the codes below and you'll be amazed by the power and easy use of our ColossalAI. Accompany this notebook with the [video tutorial](https://drive.google.com/drive/folders/1FUu74wi6FTSfpNows-prvq3MFz53NlRf?usp=sharing) for more hands-on demonstrations. 

Also, our `README.md` in this folder also contains expected result screenshots which you may use to verify your outcomes.

### Environment Setup

**[Caution]** this notebook can only be run with multiple GPUs due to parallelism settings. If you still want to run with one GPU, change config files to not use any parallelism explicitly.

To run our examples smoothly, you need to have `conda` or `miniconda` installed on your device. Check out this [website](https://docs.conda.io/en/latest/miniconda.html) for more instructions.

In [ ]:
%%sh
ls ./tutorial/  # checkout list of examples

auto_parallel
download_cifar10.py
fastfold
hybrid_parallel
large_batch_optimizer
opt
README.md
requirements.txt
sequence_parallel


Cloning into 'ColossalAI'...


In [ ]:
# Skip below if environment has been established via ../Dockerfile
%%sh
# conda create -n demo python=3.8
# conda activate demo
# pip install torch==1.12.1+cu113 torchvision==0.13.1+cu113 torchaudio==0.12.1 --extra-index-url https://download.pytorch.org/whl/cu113
# pip install colossalai==0.1.11rc3+torch1.12cu11.3 -f https://release.colossalai.org

In [ ]:
# Check versions of relevant packages
!colossalai check -i

### Large-Scale Optimizer

In [ ]:
%cd ./tutorial/large_batch_optimizer

/content/ColossalAI/examples/tutorial/large_batch_optimizer


In [ ]:
# List of files in the ColossalAI/examples/tutorial/large_batch_optimizer directory
!ls

config.py  README.md  requirements.txt	test_ci.sh  train.py


In [ ]:
!cat config.py

from colossalai.amp import AMP_TYPE

# hyperparameters
# BATCH_SIZE is as per GPU
# global batch size = BATCH_SIZE x data parallel size
BATCH_SIZE = 512
LEARNING_RATE = 3e-3
WEIGHT_DECAY = 0.3
NUM_EPOCHS = 2
WARMUP_EPOCHS = 1

# model config
NUM_CLASSES = 10

fp16 = dict(mode=AMP_TYPE.NAIVE)
clip_grad_norm = 1.0


In [ ]:
!cat train.py

import torch
import torch.nn as nn
from torchvision.models import resnet18
from tqdm import tqdm

import colossalai
from colossalai.core import global_context as gpc
from colossalai.logging import get_dist_logger
from colossalai.nn.lr_scheduler import CosineAnnealingWarmupLR
from colossalai.nn.optimizer import Lamb, Lars


class DummyDataloader():

    def __init__(self, length, batch_size):
        self.length = length
        self.batch_size = batch_size

    def generate(self):
        data = torch.rand(self.batch_size, 3, 224, 224)
        label = torch.randint(low=0, high=10, size=(self.batch_size,))
        return data, label

    def __iter__(self):
        self.step = 0
        return self

    def __next__(self):
        if self.step < self.length:
            self.step += 1
            return self.generate()
        else:
            raise StopIteration

    def __len__(self):
        return self.length


def main():
    # initialize distributed setting
    parser = colossala

In [ ]:
# Install additional requirements
!pip install -r requirements.txt

In [ ]:
# Example trial (20 steps) with lars optimizer
!colossalai run --nproc_per_node 4 train.py --config config.py --optimizer lars

/usr/local/lib/python3.8/dist-packages/torch/library.py:130: UserWarning: Overriding a previously registered kernel for the same operator and the same dispatch key
  operator: aten::index.Tensor(Tensor self, Tensor?[] indices) -> Tensor
    registered at aten/src/ATen/RegisterSchema.cpp:6
  dispatch key: Meta
  previous kernel: registered at ../aten/src/ATen/functorch/BatchRulesScatterOps.cpp:1053
       new kernel: registered at /dev/null:219 (Triggered internally at ../aten/src/ATen/core/dispatch/OperatorEntry.cpp:150.)
  self.m.impl(name, dispatch_key, fn)
/usr/local/lib/python3.8/dist-packages/torch/library.py:130: UserWarning: Overriding a previously registered kernel for the same operator and the same dispatch key
  operator: aten::index.Tensor(Tensor self, Tensor?[] indices) -> Tensor
    registered at aten/src/ATen/RegisterSchema.cpp:6
  dispatch key: Meta
  previous kernel: registered at ../aten/src/ATen/functorch/BatchRulesScatterOps.cpp:1053
       new kernel: registered at 

In [ ]:
# Now use Lamb optimizer
!colossalai run --nproc_per_node 4 train.py --config config.py --optimizer lamb

In [ ]:
# Return to parent directory
%cd ../../

/content


### Hybrid Parallelism

Example of hybriding pipeline and 1-D (2-D) tensor parallelism.

In [ ]:
%cd ./tutorial/hybrid_parallel/

/content/ColossalAI/examples/tutorial/hybrid_parallel


In [ ]:
!pip install -r requirements.txt

In [ ]:
!cat config.py  # inspect configurations

from colossalai.amp import AMP_TYPE

# hyperparameters
# BATCH_SIZE is as per GPU
# global batch size = BATCH_SIZE x data parallel size
BATCH_SIZE = 4
LEARNING_RATE = 3e-3
WEIGHT_DECAY = 0.3
NUM_EPOCHS = 2
WARMUP_EPOCHS = 1

# model config
IMG_SIZE = 224
PATCH_SIZE = 16
HIDDEN_SIZE = 128
DEPTH = 4
NUM_HEADS = 4
MLP_RATIO = 2
NUM_CLASSES = 10
CHECKPOINT = False
SEQ_LENGTH = (IMG_SIZE // PATCH_SIZE)**2 + 1    # add 1 for cls token

# parallel setting
TENSOR_PARALLEL_SIZE = 2
TENSOR_PARALLEL_MODE = '1d'

parallel = dict(
    pipeline=2,
    tensor=dict(mode=TENSOR_PARALLEL_MODE, size=TENSOR_PARALLEL_SIZE),
)

fp16 = dict(mode=AMP_TYPE.NAIVE)
clip_grad_norm = 1.0

# pipeline config
NUM_MICRO_BATCHES = parallel['pipeline']


In [ ]:
!cat train.py  # inspect training script

import os

import torch
from titans.model.vit.vit import _create_vit_model
from tqdm import tqdm

import colossalai
from colossalai.context import ParallelMode
from colossalai.core import global_context as gpc
from colossalai.logging import get_dist_logger
from colossalai.nn import CrossEntropyLoss
from colossalai.nn.lr_scheduler import CosineAnnealingWarmupLR
from colossalai.pipeline.pipelinable import PipelinableContext
from colossalai.utils import is_using_pp


class DummyDataloader():

    def __init__(self, length, batch_size):
        self.length = length
        self.batch_size = batch_size

    def generate(self):
        data = torch.rand(self.batch_size, 3, 224, 224)
        label = torch.randint(low=0, high=10, size=(self.batch_size,))
        return data, label

    def __iter__(self):
        self.step = 0
        return self

    def __next__(self):
        if self.step < self.length:
            self.step += 1
            return self.generate()
        else:
            

In [ ]:
# Execute example trial
!colossalai run --nproc_per_node 4 train.py --config config.py

In [ ]:
# Let's now tweak the configuation to adopt 2-D tensor parallelism
with open('config.py', 'r') as file:
  data = file.readlines()

# Change line 24-25
data[23] = 'TENSOR_PARALLEL_SIZE = 4\n'
data[24] = "TENSOR_PARALLEL_MODE = '2d'\n"

with open('config.py', 'w') as file:
  file.writelines(data)

In [ ]:
# New trial with hybrid of 2-D tensor parallism with pipeline parallelism 
!colossalai run --nproc_per_node 8 train.py --config config.py

config.py  README.md  requirements.txt	test_ci.sh  train.py


In [ ]:
# Return to parent directory
%cd ../../

/content


### Sequence Parallelism

Interested users may refer to [this paper](https://arxiv.org/abs/2105.13120) for implementation details.

In [ ]:
%cd ./tutorial/sequence_parallel/

/content/ColossalAI/examples/tutorial/sequence_parallel


In [ ]:
!ls

config.py  loss_func	 model	    requirements.txt  train.py
data	   lr_scheduler  README.md  test_ci.sh


In [ ]:
!pip install -r requirements.txt

In [ ]:
# Sequence parallelism is sized 2.
!cat config.py

from colossalai.amp import AMP_TYPE

# hyper-parameters
TRAIN_ITERS = 10
DECAY_ITERS = 4
WARMUP_FRACTION = 0.01
GLOBAL_BATCH_SIZE = 32    # dp world size * sentences per GPU
EVAL_ITERS = 10
EVAL_INTERVAL = 10
LR = 0.0001
MIN_LR = 1e-05
WEIGHT_DECAY = 0.01
SEQ_LENGTH = 128

# BERT config
DEPTH = 4
NUM_ATTENTION_HEADS = 4
HIDDEN_SIZE = 128

# model config
ADD_BINARY_HEAD = False

# random seed
SEED = 1234

# pipeline config
# only enabled when pipeline > 1
NUM_MICRO_BATCHES = 4

# colossalai config
parallel = dict(pipeline=1, tensor=dict(size=2, mode='sequence'))

fp16 = dict(mode=AMP_TYPE.NAIVE, verbose=True)

gradient_handler = [dict(type='SequenceParallelGradientHandler')]


In [ ]:
!cat train.py

import argparse

import torch
from data.bert_helper import SequenceParallelDataIterator, get_batch_for_sequence_parallel
from data.dummy_dataloader import DummyDataloader
from loss_func.bert_loss import BertLoss
from lr_scheduler import AnnealingLR
from model.bert import BertForPretrain, build_pipeline_bert

import colossalai
from colossalai.amp import AMP_TYPE
from colossalai.context.parallel_mode import ParallelMode
from colossalai.core import global_context as gpc
from colossalai.engine.schedule import PipelineSchedule
from colossalai.kernel import LayerNorm
from colossalai.logging import get_dist_logger
from colossalai.nn.optimizer import FusedAdam
from colossalai.utils import MultiTimer, is_using_pp


def process_batch_data(batch_data):
    tokens, types, sentence_order, loss_mask, lm_labels, padding_mask = batch_data
    if gpc.is_first_rank(ParallelMode.PIPELINE):
        data = dict(input_ids=tokens, attention_masks=padding_mask, tokentype_ids=types, lm_labels=lm_labels)
    el

In [ ]:
# Vanilla trial: sequence parallelism (2) 
!colossalai run --nproc_per_node 2 train.py -s

In [ ]:
# Let's now tweak the configuation to adopt size=2 pipeline parallelism
with open('config.py', 'r') as file:
  data = file.readlines()

# Change line 31
data[30] = "parallel = dict(pipeline=2, tensor=dict(size=2, mode='sequence'))\n"

with open('config.py', 'w') as file:
  file.writelines(data)

In [ ]:
# Run trial again: sequence parallelism (2) x pipeline parallelism (2)
!colossalai run --nproc_per_node 4 train.py -s

In [ ]:
# make sure to run before executing subsequent codes
%cd ../../

/content


### Auto Parallelism

Configuring parallism is made easier via auto-parallelism! Try out this experimental feature and watch out for its active development updates!

In [ ]:
%cd ./tutorial/auto_parallel

/content/ColossalAI/examples/tutorial/auto_parallel


In [ ]:
!ls

auto_ckpt_batchsize_test.py   bench_utils.py  requirements.txt
auto_ckpt_solver_test.py      config.py       setup.py
auto_parallel_with_resnet.py  README.md       test_ci.sh


In [ ]:
!cat auto_parallel_with_resnet.py

import torch
from torchvision.models import resnet50
from tqdm import tqdm

import colossalai
from colossalai.auto_parallel.tensor_shard.initialize import initialize_model
from colossalai.core import global_context as gpc
from colossalai.device.device_mesh import DeviceMesh
from colossalai.logging import get_dist_logger
from colossalai.nn.lr_scheduler import CosineAnnealingLR


def synthesize_data():
    img = torch.rand(gpc.config.BATCH_SIZE, 3, 32, 32)
    label = torch.randint(low=0, high=10, size=(gpc.config.BATCH_SIZE,))
    return img, label


def main():
    colossalai.launch_from_torch(config='./config.py')

    logger = get_dist_logger()

    # trace the model with meta data
    model = resnet50(num_classes=10).cuda()

    input_sample = {'x': torch.rand([gpc.config.BATCH_SIZE * torch.distributed.get_world_size(), 3, 32, 32]).to('meta')}
    device_mesh = DeviceMesh(physical_mesh_id=torch.tensor([0, 1, 2, 3]), mesh_shape=[2, 2], init_process_group=True)
    model, solution = i

In [ ]:
%%sh
pip install -r requirements.txt
conda install -c conda-forge coin-or-cbc  # dependency for strategy search

In [ ]:
# Execeute the program. It takes a short while to search for parallelization strategy.
!colossalai run --nproc_per_node 4 auto_parallel_with_resnet.py -s

In [ ]:
# Return to parent directory
%cd ../../

/content


### OPT Training Inference

There are sections for [OPT](https://arxiv.org/abs/2205.01068) finetuning and inference. This serves as a good demonstration for using ColossalAI-native [parameters](https://colossalai.org/docs/basics/colotensor_concept), [ZeRo](https://colossalai.org/docs/features/zero_with_chunk) optimizer, and [Gemini](https://colossalai.org/docs/advanced_tutorials/meet_gemini) module.

In [ ]:
# Let's first try opt fine-tuning
%cd ./tutorial/opt/opt

/content/ColossalAI/examples/tutorial/opt/opt


In [ ]:
!ls

benchmark.sh	    context.py	requirements.txt  run_clm.sh
colossalai_zero.py  README.md	run_clm.py	  run_clm_synthetic.sh


In [ ]:
!cat run_clm.py  # inspect training script

In [ ]:
!pip install datasets accelerate transformers

In [ ]:
# Run on one GPU; when cuda memory is insufficient, tensors will be offloaded to CPU 
!bash run_clm_synthetic.sh 

In [ ]:
# Train on 4 GPUs
!bash run_clm_synthetic.sh 16 0 125m 4

In [ ]:
# Now let's try inference on OPT, powered by our Energeon inference framework 
# (https://github.com/hpcaitech/EnergonAI/). 
%cd ../inference

/content/ColossalAI/examples/tutorial/opt/inference


In [ ]:
!cat README.md  # refer below for instructions

In [ ]:
# Best to run in a separate terminal 
%%sh 
docker pull hpcaitech/tutorial:opt-inference
docker run -it --rm --gpus all --ipc host -p 7070:7070 hpcaitech/tutorial:opt-inference

In [ ]:
# Run an example trial (best to run in a separate terminal as above)
!python opt_fastapi.py opt-125m --tp 2 --checkpoint /data/opt-125m

In [ ]:
%cd ../../../

### Fastfold Inference

In [ ]:
%%sh
git clone https://github.com/hpcaitech/FastFold

Cloning into 'FastFold'...


In [ ]:
%cd ./FastFold

In [ ]:
# Set up environment
%%sh
# Run the below commands in your terminal, restart the notebook with kernel named python(fastfold) 
# conda deactivate
# conda env create --name=fastfold -f environment.yml
python setup.py install

In [ ]:
# Download datasets
!bash ./scripts/download_all_data.sh data/

In [ ]:
# Run inference
!bash inference.sh

In [ ]:
%cd ../

/content


More details on fastfold (including faster kernel operations for training and data processing) can be found [here](https://github.com/hpcaitech/FastFold/).

### Stable Diffusion Inference

In [ ]:
%cd ./tutorial/stable_diffusion/

In [ ]:
# Set up environment
%%sh
# Run the below commands in your terminal, restart the notebook with kernel named python(ldm) 
# conda deactivate
# conda env create -f environment.yaml

# Install additional dependencies
pip install transformers==4.19.2 diffusers invisible-watermark
pip install pytorch-lightning

In [ ]:
# Download the model checkpoint from stable-diffusion-v2-base
%%sh
wget https://huggingface.co/stabilityai/stable-diffusion-2-base/resolve/main/512-base-ema.ckpt

In [ ]:
# See example usage of inference utility
%%sh
python scripts/txt2img.py --help 

In [ ]:
# Try stable diffusion model inference 
%%sh
python scripts/txt2img.py --prompt "a photograph of an astronaut riding a horse" --plms
    --outdir ./output \
    --ckpt 512-base-ema.ckpt \
    --config configs/inference/v2-inference.yaml  \  # there are more inference configs in the folder

In [ ]:
%cd ../../

## Conclusion

We are now finished with our tutorials. Keep updated on our [repository](https://colossalai.org/) for exciting new developments!